# Projet 10 - Modélisation

L'objectif général d'une méthode de recommandation est de *prédire les valeurs manquantes* d'une matrice utilisateur-élément, où chaque utilisateur est associé à une ligne et chaque élément à une colonne.

Ce cahier a pour objectif de présenter les étapes de conception de trois modèles de recommandation :

* Un modèle non personnalisé basé sur la popularité des articles ;
* Un modèle basé sur le contenu ;
* Un modèle basé sur le filtrage collaboratif.

# 1. Système de recommandation

Il existe deux principaux types de systèmes de recommandation : personnalisés et non personnalisés.

* Les systèmes de recommandation non personnalisés, comme les systèmes basés sur la popularité, recommandent les articles les plus populaires aux utilisateurs, par exemple les 10 meilleurs films, les livres les plus vendus ou les produits les plus achetés.
* Les systèmes de recommandation personnalisés analysent plus en détail les données des utilisateurs, leurs achats, leurs notes et leurs relations avec les autres utilisateurs. Ainsi, chaque utilisateur reçoit des recommandations personnalisées.

* Les systèmes de recommandation personnalisés les plus populaires sont le filtrage basé sur le contenu et le filtrage collaboratif.
* Les systèmes de recommandation non personnalisés, comme les systèmes basés sur la popularité, recommandent les articles les plus populaires aux utilisateurs, par exemple les 10 meilleurs films, les livres les plus vendus ou les produits les plus achetés.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import os
import pickle
from time import time
from random import randint
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 2. Préparation des données

In [2]:
src_path = Path("/home/maxime/openclassrooms/projet10_data/news-portal-user-interactions-by-globocom")
click_path = Path("/home/maxime/openclassrooms/projet10_data/news-portal-user-interactions-by-globocom/clicks/clicks")

In [3]:
def get_interaction_dataset(click_path: Path) -> pd.DataFrame:
    """Obtenir un dataset concaténé."""
    click_filelist = sorted(click_path.iterdir(), key=lambda x: x.name)
    click_list = []
    for i in range(0, len(click_filelist)):
        temp = pd.read_csv(click_path / click_filelist[i])
        temp['filename'] = i
        click_list.append(temp)
    clicks = pd.concat(click_list, axis=0, ignore_index=True)

    clicks.rename(columns={'click_article_id':'article_id'}, inplace=True)
    date2convert = ['session_start', 'click_timestamp']
    for col in date2convert:
        clicks[col] = pd.to_datetime(clicks[col], unit='ms')

    clicks.drop(columns=['click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type'], inplace=True)
    clicks = clicks.astype(np.int64)
    return clicks

clicks = get_interaction_dataset(click_path=click_path)

In [4]:
articles = pd.read_csv(src_path / 'articles_metadata.csv')

articles['created_at_ts'] = pd.to_datetime(
    (pd.to_datetime(
        articles['created_at_ts'],
        unit='ms')).dt.strftime('%Y/%m/%d'))

articles.drop(columns=['publisher_id'], inplace=True)
articles = articles.astype(np.int64)

In [5]:
with open(os.path.join(src_path,'articles_embeddings.pickle'), 'rb') as file:
    embeddings = pickle.load(file)

embeddings = embeddings.astype(np.float32)

In [6]:
# Display shape
print('Articles Dataframe shape: ', articles.shape)
print('Embedding Matrix shape: ', embeddings.shape)
print('Clicks Dataframe shape: ', clicks.shape)

Articles Dataframe shape:  (364047, 4)
Embedding Matrix shape:  (364047, 250)
Clicks Dataframe shape:  (2988181, 7)


## 2.1. Dataset d'interactions

In [7]:
print(clicks.shape)
# Display DF shape and 5 first rows
print('Clicks Dataframe shape: ', clicks.shape)
clicks.head()

clicks.to_csv('./clicks.csv', index=False)


(2988181, 7)
Clicks Dataframe shape:  (2988181, 7)


In [8]:
clicks.head()

,user_id,session_id,session_start,session_size,article_id,click_timestamp,filename
0,0,1506825423271737,1506825423000000000,2,157541,1506826828020000000,0
1,0,1506825423271737,1506825423000000000,2,68866,1506826858020000000,0
2,1,1506825426267738,1506825426000000000,2,235840,1506827017951000000,0
3,1,1506825426267738,1506825426000000000,2,96663,1506827047951000000,0
4,2,1506825435299739,1506825435000000000,2,119592,1506827090575000000,0


## 2.2. Dataset d'articles

In [9]:
# Display DF shape and 5 first rows
print('Articles Dataframe shape: ', articles.shape)
clicks[clicks["article_id"] == 157541]

articles.head()

Articles Dataframe shape:  (364047, 4)


,article_id,category_id,created_at_ts,words_count
0,0,0,1513123200000000000,168
1,1,1,1405296000000000000,189
2,2,1,1408665600000000000,250
3,3,1,1408406400000000000,230
4,4,1,1407024000000000000,162


## 2.3. Dataset Embeddings

In [10]:
df_embeddings = pd.DataFrame(embeddings)
print('Embedding Matrix shape: ', embeddings.shape)
df_embeddings.head()

Embedding Matrix shape:  (364047, 250)


,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,-0.161183,-0.957233,-0.137944,0.050855,0.830055,0.901365,-0.335148,-0.559561,-0.500603,0.165183,...,0.321248,0.313999,0.636412,0.169179,0.540524,-0.813182,0.286870,-0.231686,0.597416,0.409623
1,-0.523216,-0.974058,0.738608,0.155234,0.626294,0.485297,-0.715657,-0.897996,-0.359747,0.398246,...,-0.487843,0.823124,0.412688,-0.338654,0.320787,0.588643,-0.594137,0.182828,0.397090,-0.834364
2,-0.619619,-0.972960,-0.207360,-0.128861,0.044748,-0.387535,-0.730477,-0.066126,-0.754899,-0.242004,...,0.454756,0.473184,0.377866,-0.863887,-0.383365,0.137721,-0.810877,-0.447580,0.805932,-0.285284
3,-0.740843,-0.975749,0.391698,0.641738,-0.268645,0.191745,-0.825593,-0.710591,-0.040099,-0.110514,...,0.271535,0.036040,0.480029,-0.763173,0.022627,0.565165,-0.910286,-0.537838,0.243541,-0.885329
4,-0.279052,-0.972315,0.685374,0.113056,0.238315,0.271913,-0.568816,0.341194,-0.600554,-0.125644,...,0.238286,0.809268,0.427521,-0.615932,-0.503697,0.614450,-0.917760,-0.424061,0.185484,-0.580292


## 2.4. Valeur unique

In [11]:
print('Nombre utilisateur unique : ', clicks.user_id.nunique())
print('Nombre article unique : ', articles.article_id.nunique())
print('Nombre articles lus par les utilisateurs : ', clicks.article_id.nunique())

Nombre utilisateur unique :  322897
Nombre article unique :  364047
Nombre articles lus par les utilisateurs :  46033


# 3. Modèle de recommandation via la popularité.

Il s'agit d'un système de recommandation basé sur la popularité et/ou les tendances. Ces systèmes identifient les produits ou les films les plus populaires auprès des utilisateurs et les recommandent directement.

Par exemple, si un produit est fréquemment acheté par la plupart des utilisateurs, le système identifiera qu'il est le plus populaire. Ainsi, pour chaque nouvel utilisateur qui s'inscrit, le système le recommandera également, ce qui augmente les chances qu'il l'achète également.

In [12]:
def get_popularity_rec(clicks, n_reco=5):
    df_popularity = clicks.groupby(by=['article_id'])['click_timestamp'].count().sort_values(ascending=False).reset_index()
    df_popularity.rename(columns = {'user_id':'popularity'}, inplace=True)
    return df_popularity.article_id.head(n_reco).to_list()

In [13]:
get_popularity_rec(clicks, n_reco=5)

[160974, 272143, 336221, 234698, 123909]

# 4. Modèle basé sur la popularité

Le filtrage par contenu est un système de recommandation qui propose des articles similaires à ceux déjà consultés par l'utilisateur.

**Principe** :

* À partir des descriptions des articles choisis ou notés par un utilisateur, un profil utilisateur est construit ;
* Les articles dont les caractéristiques sont similaires à son profil sont recommandés.
* Moins un utilisateur a choisi ou noté d'articles, moins le profil obtenu est fiable, et donc les recommandations formulées sur cette base.
* Ce système repose donc sur l'historique de l'utilisateur et l'accès aux descriptions des articles est essentiel : si l'utilisateur A a lu deux articles sur les chats, le système peut lui recommander d'autres articles sur les animaux de compagnie.

**Inconvénients** :

* S'il est facile d'identifier des articles de substitution, il est évidemment difficile d'extrapoler d'un domaine à l'autre.

In [14]:
import pandas as pd
import numpy as np
from time import time
from random import randint
from sklearn.metrics.pairwise import cosine_similarity

def get_cb_reco(userID, clicks, embeddings, n_reco=5):
    '''Return 5 recommended articles ID to user'''
    print('User ID is : ', userID)
    var = clicks.loc[clicks.user_id == userID]['article_id'].to_list()
    value = var[-1]
    print('The last article read by the user is: ', value)

    emb = embeddings
    for i in range (0, len(var)):
        if i != value:
            emb = np.delete(emb, [i], 0)

    temp = np.delete(emb, [value], 0)
    distances = cosine_similarity([emb[value]], temp)[0]

    ranked_ids = np.argsort(distances)[::-1][0:n_reco]
    ranked_similarities = np.sort(distances)[::-1][0:n_reco]
    print('Recommended articles are: ')

    return ranked_ids.tolist()

In [15]:
start = time()
userID = 0
reco = get_cb_reco(userID, clicks, embeddings, n_reco=5)
print(reco)
print(f'Model execution time : {round(time() - start, 2)}s')

User ID is :  0
The last article read by the user is:  87205
Recommended articles are: 
[102720, 100020, 102412, 102611, 86703]
Model execution time : 0.85s


# 4. Modèle de filtrage collaboratif

## 4.1 Concept de bases

**Le filtrage collaboratif** est un système de recommandation qui tente de prédire les préférences d'un utilisateur en fonction des préférences similaires d'autres utilisateurs.

Cette approche repose uniquement sur la matrice utilisateur-item ; aucune connaissance intrinsèque des articles n'est nécessaire.

**Principe :**
* Proposer à un utilisateur des articles similaires à ceux qu'il a déjà choisis (ou notés) OU proposer à un utilisateur des articles choisis (ou notés) par des utilisateurs similaires.
* Ce système se base donc à la fois sur la similarité entre articles et entre utilisateurs pour formuler ses recommandations : si l'utilisateur A est similaire à l'utilisateur B, et que ce dernier a lu l'article 1, le système peut recommander l'article 1 à l'utilisateur A même s'il n'a jamais consulté d'articles similaires.

**Inconvénients :**
En revanche, s'il n'y a pas de scellement entre les domaines, la méconnaissance des caractéristiques des articles rend difficile la substitution des articles manquants.

## 4.2 Metriques d'évaluation

Il existe différentes approches pour évaluer les performances des modèles, ainsi que des indicateurs spécifiques à chaque approche :
1. **Indicateurs de notation** : Ces indicateurs permettent d’évaluer la précision d’un système de recommandation dans la prédiction des notes attribuées aux produits par les utilisateurs.
* ***Erreur quadratique moyenne (RMSE)*** : mesure de l’erreur moyenne des notes prédites.
* ***R² (R<sup>2</sup>)*** : mesure de la variation totale expliquée par le modèle.
* ***Erreur absolue moyenne (MAE)*** : similaire à la RMSE, mais utilisant la valeur absolue plutôt que le carré et la racine carrée de la moyenne.
* ***Variance expliquée*** : mesure de la variance des données expliquée par le modèle.
2. **Indicateurs de classement** : Ces indicateurs permettent d’évaluer l’intérêt des recommandations pour les utilisateurs. * ***Précision*** - mesure la proportion de produits recommandés intéressants
* ***Rappel*** - mesure la proportion de produits intéressants recommandés
* ***Gain cumulé actualisé normalisé (NDCG)*** - évalue le classement des produits recommandés à un utilisateur en fonction de leur parité
* ***Précision moyenne (MAP)*** - précision moyenne pour chaque utilisateur normalisée sur l'ensemble des utilisateurs
3. **Indicateurs de non-exactitude** : Ces indicateurs ne comparent pas les prédictions aux valeurs réelles, mais évaluent les propriétés des recommandations suivantes :
* ***Nouveauté*** - mesure la fréquence de recommandation des nouveaux produits par les utilisateurs
* ***Diversité*** - mesure la différence entre les produits d'un ensemble
* ***Sérendipité*** - mesure le degré de surprise des recommandations faites à un utilisateur spécifique en les comparant aux produits avec lesquels il a déjà interagi
* ***Couverture*** - mesure de la distribution des produits recommandés par le système

Dans notre cas, nous utiliserons des mesures de classement, à savoir : Précision, NDCG et MAP.

## 4.3 Preparation des données

Tout d'abord, il faut savoir si l'utilisateur a aimé ou non l'article qu'il a lu. Pour cela, il existe deux façons de le savoir :
1. **Explicitement** : une note (binaire ou non)
2. **Implicitement** : nombre de clics, temps passé sur la page, vidéo lue jusqu'à la fin, article parcouru jusqu'à la fin, etc.

Dans notre cas, nous utiliserons un score implicite : le nombre de clics par article pondéré par le nombre total de clics par utilisateur.

Créons une fonction qui calculera les notes implicites

In [16]:
def get_ratings(clicks):
    count_clicks_by_articles_by_user = clicks.groupby(["user_id", "article_id"]).agg(count_clicks_by_articles_by_user=("session_id", "count"))
    count_clicks_by_user = clicks.groupby(["user_id"]).agg(count_clicks_by_user=("session_id", "count"))
    clicks_count = count_clicks_by_articles_by_user.join(count_clicks_by_user, on="user_id")
    clicks_count['rating'] = clicks_count["count_clicks_by_articles_by_user"] / clicks_count["count_clicks_by_user"]
    ratings = clicks_count.reset_index().drop(["count_clicks_by_articles_by_user","count_clicks_by_user"], axis=1).rename(columns={"click_article_id":"article_id"})
    return ratings

In [17]:
ratings = get_ratings(clicks)
ratings

,user_id,article_id,rating
0,0,68866,0.125
1,0,87205,0.125
2,0,87224,0.125
3,0,96755,0.125
4,0,157541,0.125
...,...,...,...
2950705,322894,168401,0.500
2950706,322895,63746,0.500
2950707,322895,289197,0.500
2950708,322896,30760,0.500


Maintenant que nous avons la note de chaque article que l'utilisateur a lu, nous allons simplement comparer différents modèles et conserver le plus performant.

## 4.4 Échantillonnage de la base de données de notation (si nécessaire)

In [18]:
SAMPLE_RATIO = 0.005

In [19]:
ratings_sample = ratings.sample(frac=SAMPLE_RATIO, random_state=8989)
ratings_sample

,user_id,article_id,rating
2936274,317101,199393,0.500000
1620248,99395,59758,0.500000
612201,31691,293050,0.016393
1525674,90863,32750,0.076923
1100500,60312,236524,0.050000
...,...,...,...
2335685,182016,234282,0.166667
2037054,142268,214206,0.047619
1629687,100387,250118,0.052632
2916666,309688,285675,0.333333


## 4.5 Train/Test split

In [20]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(ratings_sample, train_size=0.8, random_state=31)

In [23]:
train_df.head()

,user_id,article_id,rating
1306426,74547,160974,0.111111
637456,33486,235132,0.200000
855428,47448,59400,0.022222
73521,3025,336380,0.045455
1060618,58060,57616,0.031250


In [24]:
# 1. Vérifier s'il y a AU MOINS UNE valeur manquante dans le DataFrame
if train_df.isnull().any().any():
    print("Votre DataFrame contient des valeurs manquantes.")

    # 2. Identifier les colonnes qui contiennent des valeurs manquantes
    colonnes_avec_nan = train_df.columns[train_df.isnull().any()].tolist()
    print(f"Les colonnes avec des valeurs manquantes sont : {colonnes_avec_nan}")

else:
    print("Votre DataFrame ne contient aucune valeur manquante.")

Votre DataFrame ne contient aucune valeur manquante.


## 4.6 Entrainement des modèles

In [25]:
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

from tqdm import tqdm

from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.lmf import LogisticMatrixFactorization
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, ndcg_at_k, AUC_at_k

Dans cette partie nous évaluons les 3 systèmes de recommandation.

**La factorisation matricielle logistique** est un modèle de recommandation de filtrage collaboratif qui apprend la distribution probabiliste, que l'utilisateur le veuille ou non. L'algorithme du modèle est décrit dans « Factorisation matricielle logistique pour les données de rétroaction implicite » <https://web.stanford.edu/~rezab/nips2014workshop/submits/logmat.pdf>.

**Moindres carrés alternés** est un modèle de recommandation basé sur les algorithmes décrits dans l'article « Filtrage collaboratif pour les jeux de données à rétroaction implicite », avec des optimisations de performances décrites dans « Applications de la méthode du gradient conjugué pour le filtrage collaboratif à rétroaction implicite ».

Cette fonction d'usine bascule entre les implémentations CPU et GPU trouvées dans implicit.cpu.als.AlternatingLeastSquares et implicit.gpu.als.AlternatingLeastSquares en fonction de l'indicateur use_gpu.

**Le classement bayésien personnalisé** est un modèle de recommandation qui apprend une intégration de factorisation matricielle basée sur la minimisation de la perte de classement par paires, décrite dans l'article BPR : Classement bayésien personnalisé à partir de commentaires implicites.

Cette fonction d'usine renvoie soit l'implémentation du processeur (implicit.cpu.bpr), soit l'implémentation du processeur graphique (implicit.gpu.bpr), selon la valeur de l'indicateur use_gpu.

In [30]:
def train_models(train_df, test_df, models_list, n_recommandations):
    """Evalues les 3 modèles entraînés."""
    df_results = pd.DataFrame(columns=['model', 'Precision@k','MAP@k','nDCG@k',"train_time"])
    dim = (max(train_df.user_id.max(),test_df.user_id.max())+1, max(train_df.article_id.max(),test_df.article_id.max())+1)
    train_csr = csr_matrix((train_df['rating'], (train_df['user_id'], train_df['article_id'])), dim)
    test_csr = csr_matrix((test_df['rating'], (test_df['user_id'], test_df['article_id'])), dim)
    for model in models_list:
        print("##"*30)
        print("[INFO] : Start training the model : ", model.__class__.__name__)
        train_start_time = time()
        model.fit(train_csr)
        train_time = time() - train_start_time
        precision_k = round(precision_at_k(model, train_csr, test_csr), 5)
        map_at_k = round(mean_average_precision_at_k(model, train_csr, test_csr), 5)
        ndcg_k = round(ndcg_at_k(model, train_csr, test_csr), 5)
        print("[INFO] : Precision@k = ", precision_k)
        print("[INFO] : MAP@k = ", map_at_k)
        print("[INFO] : nDCG@k = ", ndcg_k)
        print("##"*30)
        df_results = pd.DataFrame([{
            'model': model.__class__.__name__,
            'Precision@k': precision_k,
            'MAP@k': map_at_k,
            'nDCG@k': ndcg_k,
            'train_time': round(train_time,5),
        }])

    return df_results

In [31]:
models_list = [AlternatingLeastSquares(), BayesianPersonalizedRanking(), LogisticMatrixFactorization()]

train_models(
    train_df= train_df,
    test_df= test_df,
    models_list= models_list,
    n_recommandations = 5,
    )

############################################################
[INFO] : Start training the model :  AlternatingLeastSquares


100%|██████████| 2894/2894 [00:01<00:00, 1591.33it/s]


[INFO] : Precision@k =  0.00305
[INFO] : MAP@k =  0.00054
[INFO] : nDCG@k =  0.0011
############################################################
############################################################
[INFO] : Start training the model :  BayesianPersonalizedRanking


100%|██████████| 2894/2894 [00:01<00:00, 1554.15it/s]


[INFO] : Precision@k =  0.00339
[INFO] : MAP@k =  0.0012
[INFO] : nDCG@k =  0.00173
############################################################
############################################################
[INFO] : Start training the model :  LogisticMatrixFactorization


100%|██████████| 2894/2894 [00:01<00:00, 1850.29it/s]

[INFO] : Precision@k =  0.07591
[INFO] : MAP@k =  0.02366
[INFO] : nDCG@k =  0.03574
############################################################


,model,Precision@k,MAP@k,nDCG@k,train_time
0,LogisticMatrixFactorization,0.07591,0.02366,0.03574,0.72697


Le modèle le plus performant étant le LMF (Logistic Matric Factorization) avec 5,3% des articles recommandés par le système sont des articles pertinents pour l'utilisateur, donc susceptibles de lui plaire.

## 4.7 Pipeline d'entrainement

In [ ]:
import pickle

def compute_interaction_matrix(clicks):
    """Mise en place de la matrice d'interaction."""
    interactions = clicks.groupby(['user_id','article_id']).size().reset_index(name='count')

    csr_item_user = csr_matrix((interactions['count'].astype(float),
                                (interactions['article_id'],
                                 interactions['user_id'])))

    csr_user_item = csr_matrix((interactions['count'].astype(float),
                                (interactions['user_id'],
                                 interactions['article_id'])))

    return csr_item_user, csr_user_item

def get_cf_reco(clicks, userID, csr_item_user, csr_user_item, model_path=None, n_reco=5, train=True):
    """Mise en place du modèle de recommandation."""
    start = time()

    if train or model_path is None:
        model = LogisticMatrixFactorization(factors= 128, random_state=42)
        print("[INFO] : Start training model")
        model.fit(csr_user_item)

        with open('recommender.model', 'wb') as filehandle:
            pickle.dump(model, filehandle)
    else:
        with open('recommender.model', 'rb') as filehandle:
            model = pickle.load(filehandle)

    recommendations_list = []
    recommendations = model.recommend(userID, csr_user_item[userID], N=n_reco, filter_already_liked_items=True)

    print(f'[INFO] : Completed in {round(time() - start, 2)}s')
    recommendations = [elt[0] for elt in recommendations]

    return recommendations

In [33]:
userID = 61691
csr_item_user, csr_user_item = compute_interaction_matrix(clicks)
get_cf_reco(clicks, userID, csr_item_user, csr_user_item,model_path=None, n_reco=5, train=True)

[INFO] : Start training model


100%|██████████| 30/30 [00:33<00:00,  1.11s/it]


[INFO] : Completed in 34.36s


[np.int32(42770), np.float32(10.521999)]

In [34]:
get_cf_reco(clicks, userID, csr_item_user, csr_user_item, model_path="./recommender.model", n_reco=5, train=False)

[INFO] : Completed in 0.17s


[np.int32(42770), np.float32(10.521999)]